In [4]:
import pandas as pd
import os
import datetime as dt
import numpy as np
from pprint import pp

In [5]:
#use folder path as the path to folder with all the yearly storm data
folder_path='.'
dfs=[]
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Read CSV file into a dataframe
        df = pd.read_csv(file_path, low_memory=False)
        # Append dataframe to the list
        dfs.append(df)
combined_df = pd.concat(dfs, ignore_index=True)

combined_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201402,18,1000,201402,18,2000,83473.0,503953,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure developing south of Long Island a...,Eight to twelve inches of snow fell across eas...,CSV
1,201402,5,300,201402,5,2300,83491.0,504065,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure moving off the mid-Atlantic coast...,Six to twelve inches of snow fell across easte...,CSV
2,201401,18,1000,201401,19,700,82185.0,494521,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure brought a brief period of heavy s...,Four to eight inches of snow fell across easte...,CSV
3,201411,26,1000,201411,27,1000,91728.0,549746,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong coastal storm moved up the east coast...,Six to eight inches of snow fell across easter...,CSV
4,201402,13,630,201402,14,800,83476.0,503982,NEW HAMPSHIRE,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A significant winter storm brought six to twel...,Five to eight inches of snow fell across easte...,CSV


In [6]:
len(combined_df)

1671202

In [ ]:
#remove the columns we obviously do not need
combined_df=combined_df[['BEGIN_DATE_TIME','DAMAGE_PROPERTY','STATE','EVENT_TYPE']]

In [ ]:
#drop the rows with null values in the 3 columns
df=combined_df.dropna(subset=['DAMAGE_PROPERTY','BEGIN_LAT','END_LAT'], how='all')

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
#function to convert the values in 'DAMAGE_PROPERTY' to float
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

In [ ]:
df['DAMAGE_PROPERTY']=df['DAMAGE_PROPERTY'].apply(value_to_float)

In [ ]:
#function to convert values in 'BEGIN_DATE_TIME' AND 'END_DATE_TIME' to datetime variables
def convert_to_datetime(date_string):
    format_string = '%d-%b-%y %H:%M:%S'
    return dt.strptime(date_string, format_string)

In [ ]:
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].apply(convert_to_datetime)

In [ ]:
#fill the null values in 'DAMAGE_PROPERTY' column with 0s
df['DAMAGE_PROPERTY']=df['DAMAGE_PROPERTY'].fillna(0)

# State Filtering/Cleanup
***

In [ ]:
df['STATE'].unique()

1671202

In [ ]:
df['STATE'] = df['STATE'].str.upper()

df.head()

In [ ]:
df['STATE'].unique()

## Adding climate region data, dropping state data

In [ ]:
# list of US states for filtering
# hawaii and alaska dropped
# we only want the lower 48 mainland US states for data consistency when training
us_states = [ "ALABAMA", "ARIZONA", "ARKANSAS", "CALIFORNIA", "COLORADO", "CONNECTICUT", "DELAWARE", "FLORIDA", "GEORGIA", 
             "IDAHO", "ILLINOIS", "INDIANA", "IOWA", "KANSAS", "KENTUCKY", "LOUISIANA", "MAINE", "MARYLAND", "MASSACHUSETTS", 
             "MICHIGAN", "MINNESOTA", "MISSISSIPPI", "MISSOURI", "MONTANA", "NEBRASKA", "NEVADA", "NEW HAMPSHIRE", "NEW JERSEY", 
             "NEW MEXICO", "NEW YORK", "NORTH CAROLINA", "NORTH DAKOTA", "OHIO", "OKLAHOMA", "OREGON", "PENNSYLVANIA", "RHODE ISLAND", 
             "SOUTH CAROLINA", "SOUTH DAKOTA", "TENNESSEE", "TEXAS", "UTAH", "VERMONT", "VIRGINIA", "WASHINGTON", "WEST VIRGINIA", "WISCONSIN", "WYOMING"]

# dictionary of NCEI NOAA climate regions to add to our data
climate_region_dict = {
    "NORTHEAST":    ["CONNECTICUT", "DELAWARE", "MAINE", "MARYLAND", "MASSACHUSETTS", "NEW HAMPSHIRE", "NEW JERSEY", "NEW YORK", "PENNSYLVANIA", "RHODE ISLAND", "VERMONT"],
    "UPPER MIDWEST":["IOWA", "MICHIGAN", "MINNESOTA", "WISCONSIN"],
    "OHIO VALLEY":  ["ILLINOIS", "INDIANA", "KENTUCKY", "MISSOURI", "OHIO", "TENNESSEE", "WEST VIRGINIA"],
    "SOUTHEAST":    ["ALABAMA", "FLORIDA", "GEORGIA", "NORTH CAROLINA", "SOUTH CAROLINA", "VIRGINIA"],
    "NORTHERN ROCKIES AND PLAINS": ["MONTANA", "NEBRASKA", "NORTH DAKOTA", "SOUTH DAKOTA", "WYOMING"],
    "SOUTH":        ["ARKANSAS", "KANSAS", "LOUISIANA", "MISSISSIPPI", "OKLAHOMA", "TEXAS"],
    "SOUTHWEST":    ["ARIZONA", "COLORADO", "NEW MEXICO", "UTAH"],
    "NORTHWEST":    ["IDAHO", "OREGON", "WASHINGTON"],
    "WEST":         ["CALIFORNIA", "NEVADA"]
}

In [ ]:
# dropping non-states
df = df[df['STATE'].isin(us_states)]

df['STATE'].unique()

In [ ]:
df['STATE'].value_counts()

In [ ]:
formatted_region_dict = {}
for key in climate_region_dict:
    for item in climate_region_dict[key]:
        formatted_region_dict[item] = key

formatted_region_dict

In [ ]:
# key = region classification
# value = list of states
# this is probably not a very efficient way to do this
df['REGION'] = df['STATE'].map(formatted_region_dict)

df.head()

In [ ]:
df.info()

## Event Type Double Checking

In [ ]:
df["EVENT_TYPE"].value_counts()

In [ ]:
grouped = df.groupby(["EVENT_TYPE"])
grouped.sum(["DAMAGE_PROPERTY"]).drop(columns=["BEGIN_LAT", "BEGIN_LON", "END_LAT", "END_LON"]).sort_values("DAMAGE_PROPERTY", ascending=False)

In [ ]:
df["EVENT_TYPE"].unique()

In [ ]:
category_trans_dict = {
    'THUNDERSTORM WINDS/FLOODING': 'Flood',
    'HAIL/ICY ROADS': 'Hail',
    'HAIL FLOODING': 'Flood',
    'THUNDERSTORM WINDS/FLASH FLOOD': 'Flash Flood',
    'THUNDERSTORM WINDS LIGHTNING': 'Lightning',
    'THUNDERSTORM WIND/ TREES': 'Thunderstorm Wind',
    'THUNDERSTORM WIND/ TREE': 'Thunderstorm Wind',
    'THUNDERSTORM WINDS FUNNEL CLOU': 'Funnel Cloud',
    'TORNADO/WATERSPOUT': 'Waterspout',
    'THUNDERSTORM WINDS/HEAVY RAIN': 'Heavy Rain',
    'THUNDERSTORM WINDS HEAVY RAIN': 'Heavy Rain',
    'THUNDERSTORM WINDS/ FLOOD': 'Flood',
}


rest_of_cats = ['Tornado', 'Thunderstorm Wind', 'Hail', 'Winter Storm', 'Cold/Wind Chill',
       'Heavy Snow', 'Flood', 'High Wind', 'Flash Flood', 'Blizzard',
       'Ice Storm', 'Lightning', 'Frost/Freeze', 'Heavy Rain',
       'Strong Wind', 'Coastal Flood', 'Wildfire', 'Funnel Cloud',
       'Winter Weather', 'Waterspout', 'Drought', 'Debris Flow', 'Heat',
       'High Surf', 'Tropical Storm', 'Dust Devil', 'Dense Fog',
       'Hurricane (Typhoon)', 'Marine High Wind', 'Dust Storm',
       'Storm Surge/Tide', 'Lake-Effect Snow', 'Rip Current', 'Avalanche',
       'Seiche', 'Extreme Cold/Wind Chill', 'Excessive Heat', 'Tsunami',
       'Sleet', 'Freezing Fog', 'Lakeshore Flood',
       'Astronomical Low Tide', 'Tropical Depression', 'Dense Smoke',
       'Sneakerwave', 'Hurricane']

all_event_transforms = {item: item for item in rest_of_cats}
all_event_transforms.update(category_trans_dict)

pp(all_event_transforms.keys())

In [ ]:
df["EVENT_TYPE"] = df["EVENT_TYPE"].map(all_event_transforms)

df.head()

In [ ]:
# verify remaining categories
df["EVENT_TYPE"].unique()

### Some Thoughts on Event Type Consolidation
***

We may want to consider more merges including:

Hurricane (Typhoon) -> 'Hurricane'

All Floods (except maybe flash floods) -> Flood

(storm floods already filtered are a bit ambiguous in this case)

**Ideally, we either want to drop unneeded categories or run them through an unsupervised clustering algorithm to bring the total category count down to <10 for easier neural network processing**

# One-Hot Encoding, final preprocessing DF
***

In [ ]:
# double check the columns to see what we need encoded
df.head()

Columns to encode: "REGION", "EVENT_TYPE"

Could also be worthwhile to target data with only lat-long info and run clustering there, but that is its own endeavor

In [ ]:
encoded_cols_df = pd.get_dummies(df[["REGION", "EVENT_TYPE"]]).astype(int)

encoded_cols_df.head()

In [ ]:
preproc_df = pd.merge(df, encoded_cols_df, on=df.index).drop(columns=["REGION", "EVENT_TYPE", "key_0"])

preproc_df.head()

In [ ]:
output_path=#Replace with intended output path
preproc_df.to_csv(output_path)